# Chapter 2- Backpropagation

Bakpropoagation was originally discovered in the 1970's, but it wasn't until 1986 that it's importance in training neural networks was [fully realized](http://www.nature.com/nature/journal/v323/n6088/pdf/323533a0.pdf). 

## Fast Matrix-Based Approach

Let $ w^l_{jk} $ be the weight from the $ k^{\rm th} $ neuron in the $ (l - 1)^{\rm th} $ layer to the $ j^{\rm th} $ neuron in the $ l^{\rm th} $ layer. 
Let $ b^l_{j} $ for the bias of the $ j^{\rm th} $ neuron in the $ l^{\rm th} $ layer. 
Let $ a^l_{j} $ for the activation of the $ j^{\rm th} $ neuron in the $ l^{\rm th} $ layer. 

Recall that $ a^l_{j} = \sigma\left( \sum_k w^{l}_{jk} a^{l-1}_k + b^l_j \right) $ where $ k $ is the number of neurons in the $ (l - 1)^{th} $ layer.
When this equation is vectorized, it becomes $ a^l = \sigma\left(w^l a^{l-1} + b^l \right) $.
The input to the sigmoid function is a useful value, therefore it has a name, the *weighted input* and it is represented by: $ z^l \equiv w^l a^{l-1} + b^l $.

## Cost Function Assumptions

The point behind backpropagation is to compute the partial derivatives $ \partial C / \partial w $ and $ \partial C / \partial b $ of the cost function $ C $ for any weight $ w $ or bias $ b $ in the network.

### Quadratic Cost Function

Here is the quadratic cost function: $$  C = \frac{1}{2n} \sum_x \|y(x)-a^L(x)\|^2 $$
such that $ n $ is the number of training examples; $ \sum_x $ is over each individual training example, $ x $; $ y = y(x) $ is the desired output; $ L $ is the number of layers in the network; and $ a^L = a^L (x) $ is the vector of activations output from the netwrok with $ x $ as the input.

### Assumption #1

The cost function must be written as an average. 
This is necessary because with backpropagation one computes the partial derivatives $ \partial C_x / \partial w $ and $ \partial C_x / \partial b $ for a single training example, and then $ \partial C / \partial w $ and $ \partial C / \partial b $ are recovered by averagin over the training examples.

### Assumption #2

The cost function must be written as a function of the outputs from the neural network.
$ cost \ C = C\left( a^L \right) $ such that $ a^L $ is the activations of the layer $ L $.

## Hadamard Product, $ s \odot t $

Let $ s $ and $ t $ be vectors of the same dimension, the $ s \odot t $ is the *elementwise* product of the two vectors, which means the $ j^{th} $ element of $ s $ is multiplied with the $ j^{th} $ element of $ t $. 

## Equations Behind Backpropagation

#### Defining $ \delta^l_j $, *error*

The ultimate goal of backpropagation is to understand how changing the weights and biases in a network affects the cost function, which is done by computing the partial derivatives $ \partial C / \partial w^l_{jk} $ and $ \partial C / \partial b^l_{j} $.
In order to compute those partial derivatives, we need $ \delta^l_j $, which is called the *error* in the $ j^{th} $ neuron in the $ l^{th} $ layer. 

In order to understand $ \delta^l_j $, imagine that the input of each neuron, $ z^l_j $, is changed by a small amount, $ \Delta z^l_j $.
Thus, instead of the neuron outputting $ \sigma \left( z^l_j \right) $ it outputs $ \sigma \left( z^l_j + \Delta z^l_j \right) $.
Therefore, throughout the entire network, the overal change is $ \frac{\partial C}{\partial z^l_j} \Delta z^l_j $.
This in essence adds an aditional parameter to each neuron in order to better tune the outputs.
With that explanation, we can now define the *error*, $ \delta^l_j \equiv \frac{\partial C}{\partial z^l_j} $.

### Equation #1, Error in the Output Layer, $ \delta^L $

Let's begin with: $ \delta^L_j = \frac{\partial C}{\partial a^L_j} \sigma' \left( z^L_j \right) $, where $ \partial C / \partial a^L_j $ is how fast the cost is changing with the $ j^{th} $ output activation as the input, and $ \sigma' \left( z^L_j \right) $ is how fast the activation function $ \sigma $ is changing at $ z^L_j $. 

The equation in matrix-based form is $ \delta^L = \nabla_a C \odot \sigma'(z^L) $, where $ \nabla_a C $ is a vector with each element contining the partial derivatives $ \partial C / \partial a^L_j $.
Furthermore, this function can be represented as $ \delta^L = (a^L - y) \odot \sigma'(z^L) $.

### Equation #2, Error $ \delta^l $ in terms of the error in the next layer, $ \delta^{l+1} $

The equation is $ \delta^l = ((w^{l+1})^T \delta^{l+1}) \odot \sigma'(z^l) $, where $ (w^{l+1})^T $ is the transpose of the weight matrix $ w^{l+1} $ for the $ (l + 1)^{th} $ layer.
This equation essentially moves the error *backwards* through the network, because it takes the error of the next layer as input and returns the error of the previous layer. 

This equation is powerful when combined with Equation #1, because Equation #1 can compute the error for any layer in the network, then Equation #2 will calculate the remain errors of the previous layers. 

### Equation #3, Rate of Change of the Cost with Respect to any Bias, $ \frac{\partial C}{\partial b^l_j} $

The equation is: $ \frac{\partial C}{\partial b^l_j} = \delta^l_j $. 
The error, $ \delta^l_j $, is exactly equal to the rate of change, $ \partial C / \partial b^l_j $. 
This equation in shorthand is: $ \frac{\partial C}{\partial b} = \delta $, where it is assumed that $ \delta $ is the error for the same neuron as $ b $, the bias.

### Equation #4, Rate of Change of the Cost with Respect to any Weight, $ \frac{\partial C}{\partial w^l_{jk}} $

The equation is: $ \frac{\partial C}{\partial w^l_{jk}} = a^{l-1}_k \delta^l_j $. 
Another way to write this equation is: $ \frac{\partial C}{\partial w} = a_{in} \delta_{out} $.
When $ a_{in} $ is small, the gradient term $ \partial C / \partial w $ is also small, therefore the neural network learns slowly.

### General Notes on the Equations

These equations can be applied to any activation function, not just the sigmoid function.
Using these functions, one can design an activation function so that neurons don't get *saturated*, in other words slow down learning.

## The Backpropagation Algorithm

1. Input $ x $: Set the corresponding activation $ a^1 $ for the input layer.
2. Feedforward: For each $ l = 2, 3, ... , L $ compute $ z^l = w^l a^{l-1} + b^l $ and $ a^l = \sigma(z^l) $.
3. Output error $ \delta^L $: Compute the vector $ \delta^L = \nabla_a C \odot \sigma'(z^L) $.
4. Backpropagate the error: For each $ l = L-1. L-2, ..., 2 $ compute $ \delta^l = ((w^{l+1})^T \delta^{l+1}) \odot \sigma'(z^l) $.
5. Output: The gradient of the cost function is given by $ \frac{\partial C}{\partial w^l_{jk}} = a^{l-1}_k \delta^l_j $ and $ \frac{\partial C}{\partial b^l_j} = \delta^l_j $.

Theoretically the backpropagation algorithm is performed on an individual training set, but in practice backpropagation can occur using stochastic gradient descent, which means that only a subset of the training samples are used.

In [ ]:
# The function that updates the networks weights and biases is 
# Network.update_mini_batch from the previous chapter. It is 
# also replicated here.

# WILL NOT RUN!! Demonstration purposes only.

def update_mini_batch(self, mini_batch, eta):
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]
    for x, y in mini_batch:
        delta_nabla_b, delta_nabla_w = self.backprop(x, y)
        nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
    self.weights = [w - (eta / len(mini_batch)) * nw 
                    for w, nw in zip(self.weights, nabla_w)]
    self.biases = [ b - (eta / len(mini_batch)) * nb 
                    for b, nb in zip(self.biases, nabla_b)]

In [ ]:
# The function that performs the backpropagation aglorithm is
# Network.backprop from the previous chapter. It is
# also replicated here.

# WILL NOT RUN!! Demonstration purposes only.

def backprop(self, x, y):
    # initialize
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]
    # feedforward
    activation = x
    activations = [x]
    zs = []
    for b, w in zip(self.biases, self.weights):
        z = np.dot(w, activation) + b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
    # backward pass
    delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())
    
    for l in xrange(2, self.num_layers):
        z = zs[-1]
        sp = sigmoid_prime(z)
        delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
        nabla_b[-l] = delta
        nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
    return (nabla_b, nabla_w)

def cost_derivative(self, output_activations, y):
    return (output_activations - y)

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))